In [1]:
import pandas as pd
import os
import warnings 
warnings.filterwarnings(action='ignore')
import numpy as np
import gc
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import seaborn as sns
import tensorflow as tf
import warnings
import re

In [2]:
#파일 불러오기
train = pd.read_csv('open/train_processed_500.csv', encoding = 'utf-8')
train_origin = pd.read_csv('open/train.csv', encoding = 'utf-8')
test = pd.read_csv('open/test_x.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('open/sample_submission.csv', encoding = 'utf-8')

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow import keras
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Embedding, LSTM, GlobalMaxPooling1D, Conv1D, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam

In [4]:
tf.__version__

'2.3.0'

In [5]:
ls "."

 C 드라이브의 볼륨: SSD
 볼륨 일련 번호: BCA4-98AF

 C:\Users\inhoinno\DACON\2020_DACON-Kaggle\소설저자AI 디렉터리

2020-12-04  오전 01:08    <DIR>          .
2020-12-04  오전 01:08    <DIR>          ..
2020-12-04  오전 12:40    <DIR>          .ipynb_checkpoints
2020-11-26  오후 10:42            75,612 15_lstm_glove_inhoinho_v0_1.ipynb
2020-12-02  오후 06:12            76,044 15_lstm_glove_inhoinho_v0_2_ipynb의_사본.ipynb
2020-11-26  오후 08:29            38,607 15-lstm-glove-inho.ipynb
2020-11-24  오전 10:49            33,874 Augmentation.ipynb
2020-12-01  오후 04:01    <DIR>          build
2020-12-02  오후 09:19           318,220 CNN_CV_shared.ipynb
2020-12-03  오후 10:14            98,597 CNN_CV_shared-Refactoring.ipynb
2020-11-24  오전 11:15    <DIR>          dataset
2020-11-26  오후 08:29    <DIR>          dku-kaggle-class
2020-12-04  오전 12:45           110,604 DNN_CV_shared-Refactoring.ipynb
2020-12-01  오후 04:01    <DIR>          drive
2020-11-05  오후 05:31    <DIR>          ensemble
2020-11-26  오후 11:01           874,542 Logis

In [6]:
train.describe()

,index,author,length
count,183468.000000,183468.000000,183468.000000
mean,91733.500000,2.049088,162.545567
std,52962.793931,1.285455,178.318420
min,0.000000,0.000000,4.000000
25%,45866.750000,1.000000,63.000000
50%,91733.500000,2.000000,112.000000
75%,137600.250000,3.000000,188.000000
max,183467.000000,4.000000,2454.000000


In [7]:
train['length'] = train['text'].apply(len)

In [8]:
train[train['length'] < 20]

,index,text,author,length
107,107,Tell me what it is.,0,19
219,219,I said Not at all.,0,18
418,418,I dare say she had.,1,19
829,829,I am to be neutral,2,18
873,873,And where was it,2,17
...,...,...,...,...
182119,182119,Yes said he,3,13
182148,182148,He thus continued,1,18
182210,182210,Wait up for me,1,15
182241,182241,How do you do Mr,1,18


<h2>텍스트 전처리

In [9]:

#부호를 제거해주는 함수

def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

train['text']=train['text'].apply(alpha_num)


#1. 불용어 제거 remove_stopwords
# 불용어 제거해주는 함수
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

# 불용어
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "odin",
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [10]:
#전처리 적용
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
train['text'] = train['text'].apply(alpha_num).apply(remove_stopwords)
test['text'] = test['text'].apply(alpha_num).apply(remove_stopwords)

In [11]:
train[train['length'] < 20]

,index,text,author,length
107,107,tell,0,19
219,219,said not,0,18
418,418,dare say,1,19
829,829,neutral,2,18
873,873,,2,17
...,...,...,...,...
182119,182119,yes said,3,13
182148,182148,thus continued,1,18
182210,182210,wait,1,15
182241,182241,mr,1,18


In [12]:
idx_under_20 = train[train['length'] < 20].index
train = train.drop(idx_under_20)
train.describe()

,index,author,length
count,181013.000000,181013.000000,181013.000000
mean,91514.586997,2.049146,164.537862
std,53025.418222,1.286300,178.695183
min,0.000000,0.000000,20.000000
25%,45490.000000,1.000000,65.000000
50%,91441.000000,2.000000,113.000000
75%,137458.000000,3.000000,189.000000
max,183467.000000,4.000000,2454.000000


In [13]:

train

,index,text,author,length
0,0,almost choking much much wanted say strange ex...,3,235
1,1,sister asked suppose,2,34
2,2,engaged one day walked perusing janes last let...,1,312
3,3,captain porch keeping carefully way treacherou...,4,305
4,4,mercy gentlemen flung hands dont write anyway ...,3,215
...,...,...,...,...
183463,183463,urging opinion sir odins proof extremity case ...,1,571
183464,183464,many tears unwritten contract drawn us first n...,3,536
183465,183465,express consciousness no enemy punish pain con...,3,525
183466,183466,course knows no sort good moans knows better a...,3,515


In [14]:
train = train.drop_duplicates(['text'])
train = train.reset_index()

In [15]:
train

,level_0,index,text,author,length
0,0,0,almost choking much much wanted say strange ex...,3,235
1,1,1,sister asked suppose,2,34
2,2,2,engaged one day walked perusing janes last let...,1,312
3,3,3,captain porch keeping carefully way treacherou...,4,305
4,4,4,mercy gentlemen flung hands dont write anyway ...,3,215
...,...,...,...,...,...
94973,97289,97289,diningroom soon joined busily engaged separate...,1,150
94974,97290,97290,one came books toilette,1,56
94975,97291,97291,faces however tolerably calm no change visible...,1,252
94976,97292,97292,still ticking hall,2,29


<h2> 1. 토큰화

One-hot, 해싱, Bow(< 이건 RF와 LGBM에서 사용하자)

[파이썬 라이브러리를 활용한 머신러닝] 7.3장 : 텍스트 데이터를 BOW로 표현하기 

In [16]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np 

In [17]:
maxlen = 500 #100개 이후 단어는 버려
max_words = 50000 #데이터셋에서 가장 빈도 높은 1만개의 단어만 사용

In [18]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train['text'])
sequences = tokenizer.texts_to_sequences(train['text']) 

In [19]:
word_index = tokenizer.word_index
print('%s개의 고유한 토큰 확인'%len(word_index))

47151개의 고유한 토큰 확인


In [20]:
data = pad_sequences(sequences, maxlen= maxlen)
print('데이터 텐서 크기',data.shape)

데이터 텐서 크기 (94978, 500)


In [21]:
y_train = train['author']

<h2> 2. 벡터화 (임베딩)

In [22]:
glove_dir = './dataset/glove.6B/'
embeddings_index={}
f = open(os.path.join(glove_dir,'glove.6B.100d.txt'), encoding ="utf8")

for line in f : 
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word]=coefs
f.close()

In [23]:
print('%s개의 단어 벡터 확인 '%len(embeddings_index))

400001개의 단어 벡터 확인 


In [24]:
word_index = tokenizer.word_index

In [26]:
#Glove 단어 임베딩 행렬 준비하기
#임베딩 전처리
#임베딩 행렬 준비하기
embedding_dim=100
#global embedding_matrix
embedding_matrix = np.zeros((max_words, embedding_dim))

for word, i in word_index.items():
    if i< max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None :
            embedding_matrix[i] = embedding_vector# 임베딩 인덱스에 없는 단어는 모두 0이 됨.

<h2> 모델 정의하기

In [27]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense,GlobalAveragePooling2D
from keras.layers import LSTM, Activation,Dropout

In [28]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.19103999,  0.17601   ,  0.36919999, ..., -0.59680003,
         0.080843  ,  0.27866   ],
       [-0.13128   , -0.45199999,  0.043399  , ..., -0.30526   ,
        -0.045495  ,  0.56509   ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

<h3> 모델 설계

In [64]:
n_class = 5
from keras.metrics import *
from keras.layers import *
def get_base_model():
    #model = DNNClassifier(**kargs)

    model = Sequential([
        Embedding(max_words, embedding_dim, weights = [embedding_matrix], trainable =False ,input_length=maxlen),
        GlobalAveragePooling1D(),
        Dense(1024, activation='relu', ),
        Dropout(0.15),
        Dense(256, activation = 'relu'),
        Dropout(0.15),
        Dense(64, activation = 'relu'),
        Dropout(0.3),
        Dense(16, activation = 'relu'),
        Dropout(0.3),
        Dense(5, activation='softmax'),
        Dense(n_class, activation='softmax')
    ])
    model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(learning_rate=.01),
              metrics=['accuracy'])
    
    return model

In [65]:
demo = get_base_model()

In [66]:
from keras_sequential_ascii import keras2ascii

keras2ascii(demo)

           OPERATION           DATA DIMENSIONS   WEIGHTS(N)   WEIGHTS(%)

               Input   #####         500
           Embedding   emb | -------------------   5000000    92.9%
                       #####    500  100
GlobalAveragePooling1D   Y^avg -------------------         0     0.0%
                       #####         100
               Dense   XXXXX -------------------    103424     1.9%
                relu   #####        1024
             Dropout    | || -------------------         0     0.0%
                       #####        1024
               Dense   XXXXX -------------------    262400     4.9%
                relu   #####         256
             Dropout    | || -------------------         0     0.0%
                       #####         256
               Dense   XXXXX -------------------     16448     0.3%
                relu   #####          64
             Dropout    | || -------------------         0     0.0%
                       #####          64
           

<h3>모델 훈련과 평가

In [67]:
"""model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(model.summary())"""

"model.compile(loss='sparse_categorical_crossentropy',\n              optimizer='adam',\n              metrics=['accuracy'])\nprint(model.summary())"

In [68]:
X_train = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
y_train = np.array([x for x in train['author']])

In [69]:
padding_type='post'
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=maxlen)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=maxlen)

In [70]:
"""num_epochs = 20
history = model.fit(train_padded, y_train, 
                    epochs=num_epochs, verbose=2, 
                    validation_split=0.2)"""

'num_epochs = 20\nhistory = model.fit(train_padded, y_train, \n                    epochs=num_epochs, verbose=2, \n                    validation_split=0.2)'

In [71]:
n_fold = 5
n_class = 5
seed = 777
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [73]:
from keras.utils import to_categorical 
from tensorflow.python.keras.callbacks import ModelCheckpoint

trn = train_padded
tst = test_padded
y = y_train
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))

for i, (i_trn, i_val) in enumerate(cv.split(trn,y), 1):
    print(f'training model for CV #{i}')
    
    clf = get_base_model()
    
    es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=2,
                       verbose=0, mode='min', baseline=None, restore_best_weights=True)
    


    clf.fit(trn[i_trn], 
            y[i_trn],
            validation_data=(trn[i_val],y[i_val]),
            epochs=20,
            batch_size = 64,
            callbacks=[es],
           verbose = 1)
    p_val[i_val, :] = clf.predict(trn[i_val])
    p_tst += clf.predict_proba(tst) / n_fold
    del(clf)
    gc.collect()
    tf.keras.backend.clear_session()


training model for CV #1
Epoch 1/20
1188/1188 [==============================] - 10s 8ms/step - loss: 1.5737 - accuracy: 0.2985 - val_loss: 1.5725 - val_accuracy: 0.3003
Epoch 2/20
1188/1188 [==============================] - 10s 8ms/step - loss: 1.5729 - accuracy: 0.3002 - val_loss: 1.5718 - val_accuracy: 0.3003
Epoch 3/20
1188/1188 [==============================] - 10s 8ms/step - loss: 1.5730 - accuracy: 0.3003 - val_loss: 1.5720 - val_accuracy: 0.3003
training model for CV #2
Epoch 1/20
1188/1188 [==============================] - 9s 7ms/step - loss: 1.5738 - accuracy: 0.2989 - val_loss: 1.5734 - val_accuracy: 0.3003
Epoch 2/20
1188/1188 [==============================] - 8s 7ms/step - loss: 1.5728 - accuracy: 0.3003 - val_loss: 1.5727 - val_accuracy: 0.3003
Epoch 3/20
1188/1188 [==============================] - 8s 7ms/step - loss: 1.5728 - accuracy: 0.3003 - val_loss: 1.5729 - val_accuracy: 0.3003
training model for CV #3
Epoch 1/20
1188/1188 [==============================] - 10

In [ ]:
from sklearn.metrics import log_loss

In [ ]:
print(f'Accuracy (CV): {accuracy_score(y, np.argmax(p_val, axis=1)) * 100:8.4f}%')
print(f'Log Loss (CV): {log_loss(pd.get_dummies(y), p_val):8.4f}')

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
loss = history.history['loss']

val_acc  = history.history['val_accuracy']
val_loss = history.history['val_loss']
epochs = range(1,len(acc)+1)


plt.plot(epochs, acc , 'bo', label = 'Training acc')
plt.plot(epochs, val_acc, 'b', label = 'Validation acc')
plt.title('Training and validation accuracy')
plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label= 'Validation loss')
plt.title('Training and validation loss')
plt.show()


<h2> Logistic Regression

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [ ]:
scores = cross_val_score(LogisticRegression(), X_train, y_train,cv=5)
print("cv test score : {:/2f}".format(np.mean(scores)))

In [ ]:
scores = cross_val_score(LogisticRegression(), X_train_origin, y_train_origin,cv=5)
print("cv test score : {:/2f}".format(np.mean(scores)))